# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на тарифы из прошлого проекта. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Построение модели с максимально большим значением *accuracy*. По крайней мере до 0.75 на тестовой выборке.

## 1. Открытие и изучение файла

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()
df.head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


***Можно сделать вывод, что:***
- пропусков нет 
- типы данных верные 
- обучение можно делать по последнему столбцу is_ultra

## 2. Разбеение данных на выборки

In [3]:
from sklearn.model_selection import train_test_split



# Разделение обучающей выборки и валидационной
train, test_valid_features = train_test_split(df, test_size=40, random_state=12345)
test, valid = train_test_split(test_valid_features, test_size=20, random_state=12345)


# переменные для признаков и целевого признака
features_train = train.drop(['is_ultra'], axis=1)
target_train = train['is_ultra']

features_valid = valid.drop(['is_ultra'], axis=1)
target_valid = valid['is_ultra'] 

features_test = test.drop(['is_ultra'], axis=1)
target_test = test['is_ultra']

***Данные я разделили в соотношение 60-20-20.***
- для обучения 60%
- валидационная и тестовая выбоки по 20
- обучаться модель будет по последнему столбцу

## 3. Исследование моделей

In [4]:
%%time

best_model = None
best_result = 0
for depth in range(1, 10): # глубина дерева
    model1 = DecisionTreeClassifier(random_state=12345, max_depth = depth) 
    model1.fit(features_train, target_train) # обучение модели
    predictions1 = model1.predict(features_valid) # Предсказания 
    result1 = accuracy_score(target_valid, predictions1)  # Проверка
    if result1 > best_result:
        best_model = model1 # лучшая модель 
        best_result = result1 # лучшая accuracy
        
print("Accuracy:", best_result)
print("Лучшая модель:", best_model)

Accuracy: 0.8
Лучшая модель: DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')
CPU times: user 90.9 ms, sys: 0 ns, total: 90.9 ms
Wall time: 118 ms


***Модель Дерево решений***
- Модель дает хороший результат, но очень легко переобучается
- скорость работы средняя
- оптимальная глубина дерева это 7 


In [10]:
%%time

best_model = None
best_result = 0
for est in range(1, 20, 1):
    model2 = RandomForestClassifier(random_state=12345, n_estimators=est)
    model2.fit(features_train, target_train)
    predictions2 = model2.predict(features_valid)
    result2 = accuracy_score(target_valid, predictions2) 
    if result2 > best_result:
        best_model = model2# наилучшая модель
        best_result = result2#  наилучшее значение метрики accuracy на валидационных данных

print("Accuracy:", best_result)
print("Лучшая модель:", best_model)

Accuracy: 0.8
Лучшая модель: RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=5,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)
CPU times: user 1.54 s, sys: 15.9 ms, total: 1.56 s
Wall time: 1.57 s


***Случайный лес***
- очень долго выполняется 
- результат наилучший 
- отптимальное количество деревьев 5


In [11]:
%%time

model3 = LogisticRegression(random_state=12345) # модель логистической регрессии с параметром random_state=12345
model3.fit(features_train, target_train) # тренировка модели
predictions3 = model3.predict(features_valid)
result3 = accuracy_score(target_valid, predictions3) # метрика качества модели на валидационной выборке

print("Accuracy:", result3)


Accuracy: 0.75
CPU times: user 15.1 ms, sys: 3.99 ms, total: 19.1 ms
Wall time: 17.2 ms


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


***Логическая регрессия***
- самая быстрая 
- не очень хороший показатель Accuracy. Даже нет 75%

## 4. Проверка моделей на тестовой выборке

In [12]:
# Accuracy на тестовой выборке
result1 = model1.score(features_test, target_test)
result2 = model2.score(features_test, target_test)
result3 = model3.score(features_test, target_test)
print(result1)
print(result2)
print(result3)

0.85
0.95
0.85


***После проверки на тестовой выборке можно сделать вывод что:***
- Самая точная модель это случайный лес
- На втором месте Дерево решений
- И самая быстрая, но не точная это модель регресии 

## 5. Проверьте моделей на адекватность

Для проверки класификационной модели нужно сравнить результат Accuracy с случайным значением.Это 50%
***В нашем случае это: ***
- 85% - дерево  
- 95% - лес
- 85% - регрессия 
 - Все 3 модели одыкватны и выпоняют свою функцию. Хоть и делают это не с 100% точностью.

## Вывод 

Самая лучшая модель это Случайный лес. Хоть и выпоняется долго, но имеет лучшую точность.
Остальные модели тоже подойдут. Допустим дерево решений получает хороший результат,
но если переборщить с количеством деревьев можно очень легко ее преобучить.
Регрессия выводит самый плохой результат, но лучше чем 50%. В этой модели страдает точность за счет скорости ее выпонения.